<a href="https://colab.research.google.com/github/tivanello/fase5/blob/main/notebooks/Entrega%20DATATHON%20Fase5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importação da base diretamente do GitHub (com 3 abas)**

Este bloco lê o arquivo Excel que está no seu repositório GitHub (pasta **data/raw/**) e importa as três abas (**PEDE2022**, **PEDE2023**, **PEDE2024**) em dataframes separados. Depois, adiciona a coluna **ano_base** para identificar o ano de cada registro e junta tudo no dataframe final **df_fase5**.

**O que cada parte faz:**

1) **Monta a URL “raw” do GitHub**  
- **base** aponta para a pasta do arquivo no GitHub via **raw.githubusercontent.com** (é o formato certo para o Python baixar o arquivo).  
- **nome** é o nome do arquivo exatamente como está no repositório.

2) **Trata espaços no nome do arquivo**  
- **quote(nome)** converte espaços e caracteres especiais para um formato aceito em URL (por exemplo, espaço vira **%20**).  
- Isso evita erro de “arquivo não encontrado” quando o nome tem espaços.

3) **Confere as abas do Excel antes de ler**  
- **pd.ExcelFile(url)** abre o arquivo para inspecionar a estrutura.  
- **xls.sheet_names** imprime a lista de abas encontradas, para você confirmar os nomes exatos.

4) **Lê as três abas**  
- **pd.read_excel(url, sheet_name="PEDE2022")** (e equivalentes) cria **df_2022**, **df_2023**, **df_2024**.  
- Se o nome da aba estiver diferente (maiúsculas/minúsculas, espaços), ajuste o **sheet_name** usando o que apareceu em **Abas encontradas**.

5) **Cria a coluna ano_base**  
- Adiciona o ano correspondente em cada dataframe para manter rastreabilidade depois de juntar.

6) **Concatena em um único dataframe**  
- **pd.concat([df_2022, df_2023, df_2024], ignore_index=True)** junta tudo em **df_fase5** e recria o índice do zero.

7) **Validação rápida**  
- Imprime o **shape** de cada ano e do dataframe final.  
- Mostra as primeiras linhas (**head()**) para confirmar que a importação ficou correta.



In [20]:
import pandas as pd
from urllib.parse import quote

base = "https://raw.githubusercontent.com/tivanello/fase5/main/data/raw/"
nome = "BASE DE DADOS PEDE 2024 - DATATHON.xlsx"

url = base + quote(nome)

# Conferir abas
xls = pd.ExcelFile(url)
print("Abas encontradas:", xls.sheet_names)

# Ler abas (ajuste se tiver diferença de maiúsculas/minúsculas)
df_2022 = pd.read_excel(url, sheet_name="PEDE2022")
df_2023 = pd.read_excel(url, sheet_name="PEDE2023")
df_2024 = pd.read_excel(url, sheet_name="PEDE2024")

# Tag de ano (pra não virar bagunça depois)
df_2022["ano_base"] = 2022
df_2023["ano_base"] = 2023
df_2024["ano_base"] = 2024

# Junta tudo
df_fase5 = pd.concat([df_2022, df_2023, df_2024], ignore_index=True)

print("Shapes:", df_2022.shape, df_2023.shape, df_2024.shape)
print("df_fase5 shape:", df_fase5.shape)
display(df_fase5.head())



Abas encontradas: ['PEDE2022', 'PEDE2023', 'PEDE2024']
Shapes: (860, 43) (1014, 49) (1156, 51)
df_fase5 shape: (3030, 64)


,RA,Fase,Turma,Nome,Ano nasc,Idade 22,Gênero,Ano ingresso,Instituição de ensino,Pedra 20,...,Fase Ideal,Defasagem,Destaque IPV.1,INDE 2024,Pedra 2024,Avaliador5,Avaliador6,Escola,Ativo/ Inativo,Ativo/ Inativo.1
0,RA-1,7,A,Aluno-1,2003.0,19.0,Menina,2016,Escola Pública,Ametista,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RA-2,7,A,Aluno-2,2005.0,17.0,Menina,2017,Rede Decisão,Ametista,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RA-3,7,A,Aluno-3,2005.0,17.0,Menina,2016,Rede Decisão,Ametista,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RA-4,7,A,Aluno-4,2005.0,17.0,Menino,2017,Rede Decisão,Ametista,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RA-5,7,A,Aluno-5,2005.0,17.0,Menina,2016,Rede Decisão,Ametista,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df_fase5.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3030 entries, 0 to 3029
Data columns (total 64 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   RA                     3030 non-null   object 
 1   Fase                   3030 non-null   object 
 2   Turma                  3030 non-null   object 
 3   Nome                   860 non-null    object 
 4   Ano nasc               860 non-null    float64
 5   Idade 22               860 non-null    float64
 6   Gênero                 3030 non-null   object 
 7   Ano ingresso           3030 non-null   int64  
 8   Instituição de ensino  3029 non-null   object 
 9   Pedra 20               754 non-null    object 
 10  Pedra 21               1061 non-null   object 
 11  Pedra 22               1932 non-null   object 
 12  INDE 22                1932 non-null   float64
 13  Cg                     860 non-null    float64
 14  Cf                     860 non-null    float64
 15  Ct  

In [24]:
###############################################################################################################################################
# BLOCO: Conferência pós-importação (sanidade do df_fase5)
#
# O que eu faço aqui:
# - Confirmo se a concatenação das abas ficou consistente (quantidade de linhas por ano_base).
# - Confiro quantos alunos únicos (RA) existem por ano_base.
# - Verifico se há RA duplicado dentro do mesmo ano (isso atrapalha EDA/modelagem).
# - Identifico colunas 100% vazias (lixo de exportação do Excel).
# - Aponto colunas “suspeitas” que estão como texto (object), mas deveriam ser número ou data,
#   e mostro uma amostra dos valores para entender o padrão antes de corrigir.
###############################################################################################################################################

# 1) Linhas por ano + RAs únicos
print("Linhas por ano_base:")
print(df_fase5["ano_base"].value_counts(dropna=False).sort_index())

print("\nRAs únicos por ano_base:")
print(df_fase5.groupby("ano_base")["RA"].nunique())

# 2) Duplicidade de RA por ano
dup = df_fase5.duplicated(subset=["ano_base", "RA"], keep=False)
print("\nLinhas com RA duplicado dentro do mesmo ano:", int(dup.sum()))
if dup.any():
    display(df_fase5.loc[dup, ["ano_base", "RA", "Turma"]].sort_values(["ano_base","RA"]).head(20))

# 3) Colunas 100% vazias
zero_non_null = [c for c in df_fase5.columns if df_fase5[c].notna().sum() == 0]

print("Qtd de colunas 100% vazias:", len(zero_non_null))
print("Colunas 100% vazias:")
for c in zero_non_null:
    print("-", c)

# 4) Tipos suspeitos (object)
suspeitas = [c for c in df_fase5.columns
             if ("INDE" in c or c in ["Idade","Data de Nasc","INDE 2024"])
             and df_fase5[c].dtype == "object"]
print("\nColunas suspeitas (object onde deveria ser numérico/data):", suspeitas)

# 5) Amostra das suspeitas
for c in suspeitas[:6]:
    print("\nAmostra ->", c)
    display(df_fase5[c].dropna().astype(str).head(10))



Linhas por ano_base:
ano_base
2022     860
2023    1014
2024    1156
Name: count, dtype: int64

RAs únicos por ano_base:
ano_base
2022     860
2023    1014
2024    1156
Name: RA, dtype: int64

Linhas com RA duplicado dentro do mesmo ano: 0
Qtd de colunas 100% vazias: 1
Colunas 100% vazias:
- Destaque IPV.1

Colunas suspeitas (object onde deveria ser numérico/data): ['Data de Nasc', 'Idade', 'INDE 2024']

Amostra -> Data de Nasc


,Data de Nasc
860,6/17/2015
861,5/31/2014
862,2/25/2016
863,2015-12-03 00:00:00
864,11/13/2014
865,2016-10-02 00:00:00
866,6/29/2015
867,2015-08-11 00:00:00
868,1/15/2015
869,10/20/2014



Amostra -> Idade


,Idade
860,8
861,9
862,7
863,1900-01-08 00:00:00
864,8
865,1900-01-07 00:00:00
866,8
867,1900-01-07 00:00:00
868,8
869,9



Amostra -> INDE 2024


,INDE 2024
1874,7.611366666700001
1875,8.002866666700001
1876,7.952200000100001
1877,7.156366666600001
1878,5.444199999900001
1879,8.0822
1880,8.959700000000002
1881,7.346677272700001
1882,8.152624242500002
1883,7.982890909100001


In [25]:
###############################################################################################################################################
#  Copiar df_fase5 para trabalhar com segurança
#
# O que eu faço aqui:
# - Crio uma cópia do df_fase5 para não correr o risco de estragar o original sem querer.
###############################################################################################################################################

import pandas as pd
import numpy as np

df = df_fase5.copy()
print("OK. Cópia criada. Shape:", df.shape)


OK. Cópia criada. Shape: (3030, 64)


In [26]:
###############################################################################################################################################
#  Remover colunas 100% vazias (0 valores preenchidos)
#
# O que eu faço aqui:
# - Procuro colunas que não têm nenhum valor preenchido (só NaN).
# - Removo essas colunas porque são sobra/lixo do Excel e atrapalham.
###############################################################################################################################################

vazias = [c for c in df.columns if df[c].notna().sum() == 0]
print("Colunas 100% vazias (serão removidas):", vazias)

if vazias:
    df = df.drop(columns=vazias)

print("OK. Shape após remover vazias:", df.shape)


Colunas 100% vazias (serão removidas): ['Destaque IPV.1']
OK. Shape após remover vazias: (3030, 63)


In [27]:
###############################################################################################################################################
#  Ajustar Data de Nasc para datetime
#
# O que eu faço aqui:
# - Converto a coluna Data de Nasc para data (datetime).
# - Funciona mesmo com formatos mistos (ex.: 6/17/2015 e 2015-12-03 00:00:00).
# - Valores inválidos viram NaT (nulo de data).
###############################################################################################################################################

if "Data de Nasc" in df.columns:
    df["Data de Nasc"] = pd.to_datetime(df["Data de Nasc"], errors="coerce")
    print("OK. Data de Nasc ->", df["Data de Nasc"].dtype)
else:
    print("Coluna Data de Nasc não existe no df.")


OK. Data de Nasc -> datetime64[ns]


In [28]:
###############################################################################################################################################
#  Ajustar Idade para inteiro (corrigir erro típico do Excel)
#
# O que eu faço aqui:
# - Tenta converter Idade para número.
# - Se Idade estiver como data (1900-01-xx), eu extraio o 'dia' como idade (ex.: 1900-01-08 -> 8).
# - No final, Idade vira Int64 (inteiro com suporte a nulos).
###############################################################################################################################################

if "Idade" in df.columns:
    s = df["Idade"]

    idade_num = pd.to_numeric(s, errors="coerce")
    idade_dt = pd.to_datetime(s, errors="coerce")

    idade_from_date = np.where(
        idade_dt.notna() & (idade_dt.dt.year == 1900) & (idade_dt.dt.month == 1),
        idade_dt.dt.day,
        np.nan
    )

    df["Idade"] = pd.Series(idade_num).fillna(pd.Series(idade_from_date)).astype("Int64")
    print("OK. Idade ->", df["Idade"].dtype)
else:
    print("Coluna Idade não existe no df.")


OK. Idade -> Int64


In [29]:
###############################################################################################################################################
#  Ajustar INDE 2024 para float
#
# O que eu faço aqui:
# - Converto INDE 2024 para número (float).
# - Se estiver como texto, vira float; se tiver lixo, vira NaN.
###############################################################################################################################################

if "INDE 2024" in df.columns:
    df["INDE 2024"] = pd.to_numeric(df["INDE 2024"], errors="coerce")
    print("OK. INDE 2024 ->", df["INDE 2024"].dtype)
else:
    print("Coluna INDE 2024 não existe no df.")


OK. INDE 2024 -> float64


In [30]:
###############################################################################################################################################
# Conferência final + atualizar df_fase5
#
# O que eu faço aqui:
# - Mostro os tipos finais das colunas críticas.
# - Atualizo o df_fase5 para seguir o fluxo já com os dados corrigidos.
###############################################################################################################################################

for c in ["Data de Nasc", "Idade", "INDE 2024"]:
    if c in df.columns:
        print(c, "->", df[c].dtype)

df_fase5 = df
print("OK. df_fase5 atualizado. Shape:", df_fase5.shape)


Data de Nasc -> datetime64[ns]
Idade -> Int64
INDE 2024 -> float64
OK. df_fase5 atualizado. Shape: (3030, 63)


In [32]:
###############################################################################################################################################
# BLOCO: Auditoria geral de tipos (diagnóstico)
#
# O que eu faço aqui:
# - Examino todas as colunas do df_fase5 e marco como "suspeitas" quando:
#   1) A coluna é texto (object/string), mas parece numérica (muitos valores viram número após limpeza)
#   2) A coluna é texto (object/string) e tem "cara de data" pelo nome (ex.: data, nasc, nascimento),
#      e muitos valores viram datetime
#   3) A coluna é float, mas parece inteiro (quase tudo é inteiro, só ficou float por causa de NaN)
# - Eu só tento parsear data nas colunas que parecem ser de data (pra evitar aviso e custo).
###############################################################################################################################################

import pandas as pd
import numpy as np
import re
import warnings

df = df_fase5.copy()

def clean_numeric_series(s):
    x = s.astype("string").str.strip()
    x = x.str.replace(r"\.", "", regex=True)     # remove pontos (milhar)
    x = x.str.replace(",", ".", regex=False)     # vírgula decimal -> ponto
    x = x.str.replace(r"[^0-9\.\-]", "", regex=True)
    return pd.to_numeric(x, errors="coerce")

def looks_like_date_col(colname):
    c = str(colname).lower()
    return any(k in c for k in ["data", "dt", "nasc", "nascimento", "date"])

def guess_profile(col):
    s = df[col]
    dtype = str(s.dtype)

    sample = s.dropna()
    if len(sample) > 500:
        sample = sample.sample(500, random_state=42)

    pct_num = None
    pct_date = None
    pct_intlike = None

    if dtype in ["object", "string"]:
        num = clean_numeric_series(sample)
        pct_num = float(num.notna().mean()) if len(sample) else 0.0

        # Só tenta data se o nome da coluna indicar que é data
        if looks_like_date_col(col):
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                dt = pd.to_datetime(sample.astype("string").str.strip(), errors="coerce")
            pct_date = float(dt.notna().mean()) if len(sample) else 0.0
        else:
            pct_date = 0.0

    if "float" in dtype:
        vals = sample.astype(float)
        pct_intlike = float(np.isclose(vals.dropna() % 1, 0).mean()) if vals.notna().any() else 0.0

    return dtype, pct_num, pct_date, pct_intlike, sample.astype("string").head(8).tolist()

suspeitas = []

for col in df.columns:
    dtype, pct_num, pct_date, pct_intlike, amostra = guess_profile(col)

    flag = False
    motivo = []

    if pct_num is not None and pct_num >= 0.85:
        flag = True
        motivo.append(f"parece_numero={pct_num:.0%}")

    # só marcamos "parece_data" se for coluna com cara de data
    if looks_like_date_col(col) and pct_date is not None and pct_date >= 0.60:
        flag = True
        motivo.append(f"parece_data={pct_date:.0%}")

    if pct_intlike is not None and pct_intlike >= 0.98:
        flag = True
        motivo.append(f"float_parece_inteiro={pct_intlike:.0%}")

    if flag:
        suspeitas.append({
            "coluna": col,
            "dtype_atual": dtype,
            "motivo": "; ".join(motivo),
            "amostra": amostra
        })

suspeitas_df = pd.DataFrame(suspeitas).sort_values(["dtype_atual","coluna"]).reset_index(drop=True)

print("Qtd colunas suspeitas:", len(suspeitas_df))
display(suspeitas_df)


Qtd colunas suspeitas: 21


,coluna,dtype_atual,motivo,amostra
0,Ano nasc,float64,float_parece_inteiro=100%,"[2013.0, 2012.0, 2006.0, 2008.0, 2009.0, 2013...."
1,Cf,float64,float_parece_inteiro=100%,"[166.0, 169.0, 33.0, 140.0, 114.0, 6.0, 75.0, ..."
2,Cg,float64,float_parece_inteiro=100%,"[604.0, 704.0, 413.0, 843.0, 649.0, 32.0, 279...."
3,Ct,float64,float_parece_inteiro=100%,"[9.0, 14.0, 5.0, 7.0, 9.0, 1.0, 4.0, 3.0]"
4,Defas,float64,float_parece_inteiro=100%,"[-1.0, -1.0, -2.0, -1.0, -1.0, -1.0, -1.0, -2.0]"
5,Defasagem,float64,float_parece_inteiro=100%,"[-1.0, -2.0, -1.0, 0.0, -1.0, -1.0, 0.0, -2.0]"
6,IAN,float64,float_parece_inteiro=98%,"[5.0, 5.0, 5.0, 10.0, 5.0, 5.0, 5.0, 5.0]"
7,Idade 22,float64,float_parece_inteiro=100%,"[9.0, 10.0, 16.0, 14.0, 13.0, 9.0, 10.0, 15.0]"
8,Nº Av,float64,float_parece_inteiro=100%,"[3.0, 3.0, 4.0, 0.0, 2.0, 4.0, 2.0, 3.0]"
9,Avaliador1,object,parece_numero=100%,"[Avaliador-21, Avaliador-13, Avaliador-22, Ava..."


In [33]:
###############################################################################################################################################
# BLOCO 1: Criar ra_num (tirar "RA-" e manter só o número) sem mexer no ra original
#
# O que eu faço aqui:
# - Extraio o número de ra (ex.: "RA-1234" -> 1234).
# - Crio a coluna ra_num como Int64 (aceita nulos).
# - Não altero a coluna ra original.
###############################################################################################################################################

import pandas as pd

df = df_fase5.copy()

if "RA" in df.columns and "ra" not in df.columns:
    # se você ainda estiver antes do snake_case
    base_col = "RA"
else:
    base_col = "ra"

df["ra_num"] = pd.to_numeric(
    df[base_col].astype("string").str.extract(r"(\d+)", expand=False),
    errors="coerce"
).astype("Int64")

print("OK. ra_num criado.")
print("Nulos em ra_num:", int(df["ra_num"].isna().sum()))
display(df[[base_col, "ra_num"]].drop_duplicates().head(15))

df_fase5 = df


OK. ra_num criado.
Nulos em ra_num: 0


,RA,ra_num
0,RA-1,1
1,RA-2,2
2,RA-3,3
3,RA-4,4
4,RA-5,5
5,RA-6,6
6,RA-7,7
7,RA-8,8
8,RA-9,9
9,RA-10,10


In [34]:
###############################################################################################################################################
# Verificar se ra_num ficou único dentro de cada ano_base
#
# O que eu faço aqui:
# - Confiro se ra_num é único por ano_base (igual você fez com ra).
# - Se não for, eu mostro exemplos (pra entender se existe reaproveitamento de RA em outro contexto).
###############################################################################################################################################

df = df_fase5.copy()

if "ano_base" in df.columns and "ra_num" in df.columns:
    dup = df.duplicated(subset=["ano_base", "ra_num"], keep=False)
    print("Linhas com ra_num duplicado dentro do mesmo ano:", int(dup.sum()))
    if dup.any():
        cols_show = [c for c in ["ano_base", "ra", "RA", "ra_num", "turma", "Turma"] if c in df.columns]
        display(df.loc[dup, cols_show].sort_values(["ano_base","ra_num"]).head(30))
else:
    print("Faltando ano_base ou ra_num no df.")


Linhas com ra_num duplicado dentro do mesmo ano: 0


In [35]:
###############################################################################################################################################
# Criar nome_num e nome_anon_num (tirar "Aluno-" e manter só o número) sem mexer nos originais
#
# O que eu faço aqui:
# - Extraio o número de "Nome" (ex.: "Aluno-715" -> 715) e guardo em nome_num.
# - Extraio o número de "Nome Anonimizado" (ex.: "Aluno-1386" -> 1386) e guardo em nome_anon_num.
# - Não altero as colunas originais.
###############################################################################################################################################

import pandas as pd

df = df_fase5.copy()

# Resolve nomes das colunas (antes/depois do snake_case)
col_nome = "nome" if "nome" in df.columns else ("Nome" if "Nome" in df.columns else None)
col_nome_anon = "nome_anonimizado" if "nome_anonimizado" in df.columns else ("Nome Anonimizado" if "Nome Anonimizado" in df.columns else None)

if col_nome:
    df["nome_num"] = pd.to_numeric(
        df[col_nome].astype("string").str.extract(r"(\d+)", expand=False),
        errors="coerce"
    ).astype("Int64")
    print("OK. nome_num criado. Nulos:", int(df["nome_num"].isna().sum()))
    display(df[[col_nome, "nome_num"]].drop_duplicates().head(10))
else:
    print("Coluna Nome/nome não encontrada.")

if col_nome_anon:
    df["nome_anon_num"] = pd.to_numeric(
        df[col_nome_anon].astype("string").str.extract(r"(\d+)", expand=False),
        errors="coerce"
    ).astype("Int64")
    print("OK. nome_anon_num criado. Nulos:", int(df["nome_anon_num"].isna().sum()))
    display(df[[col_nome_anon, "nome_anon_num"]].drop_duplicates().head(10))
else:
    print("Coluna Nome Anonimizado/nome_anonimizado não encontrada.")

df_fase5 = df


OK. nome_num criado. Nulos: 2170


,Nome,nome_num
0,Aluno-1,1
1,Aluno-2,2
2,Aluno-3,3
3,Aluno-4,4
4,Aluno-5,5
5,Aluno-6,6
6,Aluno-7,7
7,Aluno-8,8
8,Aluno-9,9
9,Aluno-10,10


OK. nome_anon_num criado. Nulos: 860


,Nome Anonimizado,nome_anon_num
0,NaN,<NA>
860,Aluno-861,861
861,Aluno-862,862
862,Aluno-863,863
863,Aluno-864,864
864,Aluno-865,865
865,Aluno-866,866
866,Aluno-867,867
867,Aluno-868,868
868,Aluno-869,869


In [36]:
###############################################################################################################################################
# Checar se nome_num e nome_anon_num são consistentes (opcional)
#
# O que eu faço aqui:
# - Verifico se existe algum caso estranho:
#   1) nome original existe, mas nome_num ficou nulo (não encontrou número)
#   2) nome_anon existe, mas nome_anon_num ficou nulo
###############################################################################################################################################

df = df_fase5.copy()

checks = []

if "nome_num" in df.columns:
    col_nome = "nome" if "nome" in df.columns else ("Nome" if "Nome" in df.columns else None)
    if col_nome:
        m = df[col_nome].notna() & df["nome_num"].isna()
        checks.append(("nome_num_nulo_com_nome_preenchido", int(m.sum())))
        if m.any():
            display(df.loc[m, [col_nome, "nome_num"]].head(10))

if "nome_anon_num" in df.columns:
    col_nome_anon = "nome_anonimizado" if "nome_anonimizado" in df.columns else ("Nome Anonimizado" if "Nome Anonimizado" in df.columns else None)
    if col_nome_anon:
        m = df[col_nome_anon].notna() & df["nome_anon_num"].isna()
        checks.append(("nome_anon_num_nulo_com_nome_anon_preenchido", int(m.sum())))
        if m.any():
            display(df.loc[m, [col_nome_anon, "nome_anon_num"]].head(10))

print("Checagens:")
for k, v in checks:
    print("-", k, "=", v)


Checagens:
- nome_num_nulo_com_nome_preenchido = 0
- nome_anon_num_nulo_com_nome_anon_preenchido = 0


In [37]:
###############################################################################################################################################
# Visão geral dos avaliadores (quantos IDs existem e quantos nulos)
#
# O que eu faço aqui:
# - Para cada coluna Avaliador*, conto:
#   1) quantos valores preenchidos
#   2) quantos nulos
#   3) quantos avaliadores distintos
###############################################################################################################################################

import pandas as pd

df = df_fase5.copy()

# resolve nomes antes/depois do snake_case
avaliador_cols = [c for c in df.columns if str(c).lower().startswith("avaliador")]

print("Colunas de avaliador encontradas:", avaliador_cols)

for c in avaliador_cols:
    n_total = len(df)
    n_nonnull = int(df[c].notna().sum())
    n_null = int(df[c].isna().sum())
    n_unique = int(df[c].dropna().astype("string").nunique())

    print("\n", c)
    print("  preenchidos:", n_nonnull, "| nulos:", n_null, "| distintos:", n_unique)


Colunas de avaliador encontradas: ['Avaliador1', 'Avaliador2', 'Avaliador3', 'Avaliador4', 'Avaliador5', 'Avaliador6']

 Avaliador1
  preenchidos: 2827 | nulos: 203 | distintos: 20

 Avaliador2
  preenchidos: 2827 | nulos: 203 | distintos: 23

 Avaliador3
  preenchidos: 2034 | nulos: 996 | distintos: 21

 Avaliador4
  preenchidos: 1051 | nulos: 1979 | distintos: 11

 Avaliador5
  preenchidos: 148 | nulos: 2882 | distintos: 5

 Avaliador6
  preenchidos: 6 | nulos: 3024 | distintos: 1


In [8]:
################################################################################################################################################
# Padronizar nomes para snake_case (sem unificar nada)
# A ideia é: transformar “Ano nasc”, “Instituição de ensino”, “Ativo/ Inativo” etc. em nomes consistentes (sem acento, sem espaços, minúsculo).
################################################################################################################################################
import unicodedata
import re

df = df_fase5.copy()

def to_snake(s):
    s = str(s).strip()
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    s = s.replace("/", "_").replace("-", "_")
    s = re.sub(r"\s+", "_", s)          # espaços -> _
    s = re.sub(r"[^a-zA-Z0-9_]", "", s) # remove resto
    s = re.sub(r"_+", "_", s)           # __ -> _
    return s.lower()

df.columns = [to_snake(c) for c in df.columns]

df_fase5 = df  # atualiza

print("OK. Colunas padronizadas para snake_case.")
print("Amostra de colunas:", list(df_fase5.columns)[:25])



OK. Colunas padronizadas para snake_case.
Amostra de colunas: ['ra', 'fase', 'turma', 'nome', 'ano_nasc', 'idade_22', 'genero', 'ano_ingresso', 'instituicao_de_ensino', 'pedra_20', 'pedra_21', 'pedra_22', 'inde_22', 'cg', 'cf', 'ct', 'no_av', 'avaliador1', 'rec_av1', 'avaliador2', 'rec_av2', 'avaliador3', 'rec_av3', 'avaliador4', 'rec_av4']


In [18]:
df_fase5.head()

,ra,fase,turma,nome,ano_nasc,idade_22,genero,ano_ingresso,instituicao_de_ensino,pedra_20,...,avaliador5,avaliador6,escola,ativo_inativo,ativo_inativo1,nota_mat,nota_port,nota_ing,inde_2022,pedra_2022
0,RA-1,7,A,Aluno-1,2003.0,19.0,Menina,2016,Escola Pública,Ametista,...,NaN,NaN,NaN,NaN,NaN,2.7,3.5,6.0,5.783,Quartzo
1,RA-2,7,A,Aluno-2,2005.0,17.0,Menina,2017,Rede Decisão,Ametista,...,NaN,NaN,NaN,NaN,NaN,6.3,4.5,9.7,7.055,Ametista
2,RA-3,7,A,Aluno-3,2005.0,17.0,Menina,2016,Rede Decisão,Ametista,...,NaN,NaN,NaN,NaN,NaN,5.8,4.0,6.9,6.591,Ágata
3,RA-4,7,A,Aluno-4,2005.0,17.0,Menino,2017,Rede Decisão,Ametista,...,NaN,NaN,NaN,NaN,NaN,2.8,3.5,8.7,5.951,Quartzo
4,RA-5,7,A,Aluno-5,2005.0,17.0,Menina,2016,Rede Decisão,Ametista,...,NaN,NaN,NaN,NaN,NaN,7.0,2.9,5.7,7.427,Ametista


In [19]:
###############################################################################################################################################
# BLOCO: Verificar colunas duplicadas e comparar se o conteúdo é igual ou diferente
#
# O que eu faço aqui:
# - Identifico nomes de colunas duplicados (ex.: fase_ideal aparecendo duas vezes).
# - Para cada nome duplicado, eu comparo as colunas entre si:
#     1) Quantos valores são diferentes (tratando NaN como "igual a NaN")
#     2) Quantos casos uma tem valor e a outra está vazia (e vice-versa)
# - Mostro uma amostra de linhas onde existe diferença, para você enxergar o padrão.
###############################################################################################################################################

def comparar_duplicadas(df_fase5, amostra=10):
    dup_names = df_fase5.columns[df_fase5.columns.duplicated()].unique().tolist()

    if not dup_names:
        print("Não existem colunas duplicadas.")
        return

    print("Colunas duplicadas encontradas:", dup_names)

    for name in dup_names:
        cols_idx = [i for i, c in enumerate(df_fase5.columns) if c == name]
        cols = [df_fase5.iloc[:, i] for i in cols_idx]

        print("\n" + "-"*120)
        print("Coluna duplicada:", name, "| ocorrências:", len(cols_idx))

        # Compara a primeira contra cada uma das outras
        base = cols[0]
        for j in range(1, len(cols)):
            other = cols[j]

            # diferença tratando NaN == NaN
            diff_mask = ~(base.fillna("__NA__").astype(str).eq(other.fillna("__NA__").astype(str)))
            diff_count = int(diff_mask.sum())

            # casos onde uma tem valor e a outra não
            base_only = int(base.notna().sum() - base[base.notna() & other.notna()].notna().sum())
            other_only = int(other.notna().sum() - other[base.notna() & other.notna()].notna().sum())

            # overlap (ambas com valor)
            both = int((base.notna() & other.notna()).sum())

            print(f"\nComparação: {name} (ocorrência 1) VS {name} (ocorrência {j+1})")
            print("Diferenças (valor):", diff_count)
            print("Ambas preenchidas:", both)
            print("Só a 1ª preenchida:", base_only)
            print("Só a 2ª preenchida:", other_only)

            if diff_count > 0:
                print("\nAmostra de linhas com diferença:")
                idx = df.index[diff_mask][:amostra]
                am = pd.DataFrame({
                    f"{name}_1": base.loc[idx].astype("string"),
                    f"{name}_{j+1}": other.loc[idx].astype("string")
                }, index=idx)
                display(am)

comparar_duplicadas(df_long, amostra=10)

Não existem colunas duplicadas.


In [9]:
###############################################################################################################################################
# BLOCO: Unificar colunas duplicadas e padronizar colunas-chave (wide)
#
# O que eu faço aqui:
# - Quando o mesmo conceito aparece com nomes diferentes (ex.: defas e defasagem), eu junto tudo em UMA coluna só.
# - Quando existe coluna duplicada (ex.: ativo_inativo e ativo_inativo_1), eu pego o primeiro valor preenchido e elimino a duplicata.
# - Também crio nomes padrão para medidas por ano (inde_2022/2023/2024 e pedra_2022/2023/2024).
# - No final, converto o que deve ser número (inde/notas/defasagem) para tipo numérico.
###############################################################################################################################################

df = df_fase5.copy()

def coalesce(new_col, candidates):
    """
    Junta várias colunas do mesmo assunto em uma só (new_col).
    Regra: pega o primeiro valor não-nulo da esquerda para a direita (bfill).
    Depois remove as colunas extras, mantendo apenas new_col.
    """
    exist = [c for c in candidates if c in df.columns]
    if not exist:
        return
    df[new_col] = df[exist].bfill(axis=1).iloc[:, 0]
    drop_cols = [c for c in exist if c != new_col]
    if drop_cols:
        df.drop(columns=drop_cols, inplace=True)

# 1) fase_ideal: unifica variações do mesmo conceito
coalesce("fase_ideal", ["fase_ideal", "fase_ideal_1"])

# 2) defasagem: junta defas + defasagem
coalesce("defasagem", ["defasagem", "defas"])

# 3) notas: padroniza notas em colunas únicas
coalesce("nota_mat", ["nota_mat", "matem", "mat"])
coalesce("nota_port", ["nota_port", "portug", "por"])
coalesce("nota_ing", ["nota_ing", "ingles", "ing"])

# 4) inde/pedra por ano: padroniza colunas específicas de cada ano
coalesce("inde_2022", ["inde_22"])
coalesce("inde_2023", ["inde_2023", "inde_23"])
coalesce("inde_2024", ["inde_2024"])

coalesce("pedra_2022", ["pedra_22"])
coalesce("pedra_2023", ["pedra_2023", "pedra_23"])
coalesce("pedra_2024", ["pedra_2024"])

# 5) ativo_inativo: junta duplicada (se existir)
coalesce("ativo_inativo", ["ativo_inativo", "ativo_inativo_1"])

# 6) garantir tipos numéricos onde faz sentido (evita ficar como texto)
for c in ["inde_2022","inde_2023","inde_2024","nota_mat","nota_port","nota_ing","defasagem"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# atualiza o dataframe principal
df_fase5 = df

print("OK. Unificações aplicadas.")
print("Colunas finais-chave que eu espero ver:")
for c in ["fase_ideal","defasagem","nota_mat","nota_port","nota_ing",
          "inde_2022","inde_2023","inde_2024",
          "pedra_2022","pedra_2023","pedra_2024",
          "ativo_inativo"]:
    print(c, "->", ("OK" if c in df_fase5.columns else "NÃO EXISTE"))



OK. Unificações aplicadas.
Colunas finais-chave que eu espero ver:
fase_ideal -> OK
defasagem -> OK
nota_mat -> OK
nota_port -> OK
nota_ing -> OK
inde_2022 -> OK
inde_2023 -> OK
inde_2024 -> OK
pedra_2022 -> OK
pedra_2023 -> OK
pedra_2024 -> OK
ativo_inativo -> OK


In [16]:
###############################################################################################################################################
# BLOCO: Verificar colunas duplicadas e comparar se o conteúdo é igual ou diferente
#
# O que eu faço aqui:
# - Identifico nomes de colunas duplicados (ex.: fase_ideal aparecendo duas vezes).
# - Para cada nome duplicado, eu comparo as colunas entre si:
#     1) Quantos valores são diferentes (tratando NaN como "igual a NaN")
#     2) Quantos casos uma tem valor e a outra está vazia (e vice-versa)
# - Mostro uma amostra de linhas onde existe diferença, para você enxergar o padrão.
###############################################################################################################################################

def comparar_duplicadas(df, amostra=10):
    dup_names = df.columns[df.columns.duplicated()].unique().tolist()

    if not dup_names:
        print("Não existem colunas duplicadas.")
        return

    print("Colunas duplicadas encontradas:", dup_names)

    for name in dup_names:
        cols_idx = [i for i, c in enumerate(df.columns) if c == name]
        cols = [df.iloc[:, i] for i in cols_idx]

        print("\n" + "-"*120)
        print("Coluna duplicada:", name, "| ocorrências:", len(cols_idx))

        # Compara a primeira contra cada uma das outras
        base = cols[0]
        for j in range(1, len(cols)):
            other = cols[j]

            # diferença tratando NaN == NaN
            diff_mask = ~(base.fillna("__NA__").astype(str).eq(other.fillna("__NA__").astype(str)))
            diff_count = int(diff_mask.sum())

            # casos onde uma tem valor e a outra não
            base_only = int(base.notna().sum() - base[base.notna() & other.notna()].notna().sum())
            other_only = int(other.notna().sum() - other[base.notna() & other.notna()].notna().sum())

            # overlap (ambas com valor)
            both = int((base.notna() & other.notna()).sum())

            print(f"\nComparação: {name} (ocorrência 1) VS {name} (ocorrência {j+1})")
            print("Diferenças (valor):", diff_count)
            print("Ambas preenchidas:", both)
            print("Só a 1ª preenchida:", base_only)
            print("Só a 2ª preenchida:", other_only)

            if diff_count > 0:
                print("\nAmostra de linhas com diferença:")
                idx = df.index[diff_mask][:amostra]
                am = pd.DataFrame({
                    f"{name}_1": base.loc[idx].astype("string"),
                    f"{name}_{j+1}": other.loc[idx].astype("string")
                }, index=idx)
                display(am)

comparar_duplicadas(df_long, amostra=10)

Colunas duplicadas encontradas: ['fase_ideal']

------------------------------------------------------------------------------------------------------------------------
Coluna duplicada: fase_ideal | ocorrências: 2

Comparação: fase_ideal (ocorrência 1) VS fase_ideal (ocorrência 2)
Diferenças (valor): 0
Ambas preenchidas: 3030
Só a 1ª preenchida: 0
Só a 2ª preenchida: 0


In [17]:
###############################################################################################################################################
# BLOCO: Remover coluna duplicada (fase_ideal) mantendo apenas uma
#
# O que eu faço aqui:
# - Confirmo quais colunas estão duplicadas.
# - Como você já validou que as duas fase_ideal são idênticas, eu simplesmente removo a duplicata.
# - No final, valido que não sobrou nenhuma coluna duplicada.
###############################################################################################################################################

# 1) Mostrar duplicadas (só pra conferir)
dup_cols = df_long.columns[df_long.columns.duplicated()].tolist()
print("Colunas duplicadas:", dup_cols)

# 2) Remover duplicadas mantendo a primeira ocorrência
df_long = df_long.loc[:, ~df_long.columns.duplicated()]

# 3) Confirmação
print("Ainda tem duplicadas?", df_long.columns.duplicated().any())
print("Shape df_long:", df_long.shape)


Colunas duplicadas: ['fase_ideal']
Ainda tem duplicadas? False
Shape df_long: (3030, 32)


In [11]:
###############################################################################################################################################
# BLOCO: Criar dataset LONG (1 linha por aluno + ano) a partir do df_fase5 padronizado
#
# O que eu faço aqui:
# - Transformo a base wide (com inde_2022/2023/2024 e pedra_2022/2023/2024) em um formato LONG.
# - Para cada linha, escolho inde e pedra conforme ano_base.
# - Crio fase_num (número da fase) para facilitar análise e modelagem.
# - Crio flags sem_inde e sem_pedra para documentar dados faltantes (não “esconde” o problema).
# - Salvo o resultado em data/processed (parquet e csv).
###############################################################################################################################################

import os
import pandas as pd
import numpy as np

df = df_fase5.copy()

def pick_ano(dfx, ano_col, col_2022, col_2023, col_2024):
    """
    Cria uma coluna 'do ano' escolhendo qual coluna usar conforme ano_base.
    """
    out = pd.Series([pd.NA] * len(dfx), index=dfx.index, dtype="object")

    m22 = dfx[ano_col] == 2022
    m23 = dfx[ano_col] == 2023
    m24 = dfx[ano_col] == 2024

    if col_2022 in dfx.columns:
        out.loc[m22] = dfx.loc[m22, col_2022]
    if col_2023 in dfx.columns:
        out.loc[m23] = dfx.loc[m23, col_2023]
    if col_2024 in dfx.columns:
        out.loc[m24] = dfx.loc[m24, col_2024]

    return out

# 1) Criar inde e pedra “do ano”
df["inde"] = pick_ano(df, "ano_base", "inde_2022", "inde_2023", "inde_2024")
df["pedra"] = pick_ano(df, "ano_base", "pedra_2022", "pedra_2023", "pedra_2024")

df["inde"] = pd.to_numeric(df["inde"], errors="coerce")

# 2) Criar fase_num a partir de fase (ex.: FASE 2, 1A, 7E, ALFA)
df["fase_num"] = pd.to_numeric(
    df["fase"].astype(str).str.extract(r"(\d+)", expand=False),
    errors="coerce"
).astype("Int64")

# 3) Selecionar colunas relevantes para o LONG
cols_long = [
    "ra","ano_base","fase","fase_num","turma","genero","instituicao_de_ensino",
    "inde","pedra",
    "iaa","ieg","ips","ida","ipv","ian","ipp",
    "nota_mat","nota_port","nota_ing",
    "defasagem","fase_ideal",
    "ativo_inativo",
    "rec_psicologia","rec_av1","rec_av2","rec_av3","rec_av4",
    "destaque_ieg","destaque_ida","destaque_ipv"
]
cols_long = [c for c in cols_long if c in df.columns]

df_long = df[cols_long].copy()

# 4) Flags de qualidade (para relatório e decisão de modelagem)
df_long["sem_inde"] = df_long["inde"].isna()
df_long["sem_pedra"] = df_long["pedra"].isna()

# 5) Blindagem parquet: textos -> string
obj_cols = df_long.select_dtypes(include=["object"]).columns.tolist()

for c in obj_cols:
    df_long[c] = df_long[c].astype("string")

# 6) Salvar em data/processed
os.makedirs("data/processed", exist_ok=True)
df_long.to_parquet("data/processed/pede_long_2022_2024.parquet", index=False)
df_long.to_csv("data/processed/pede_long_2022_2024.csv", index=False, encoding="utf-8")

print("OK. LONG salvo em data/processed/")
print("Shape:", df_long.shape)

print("\nNulos de inde por ano:")
print(df_long.groupby("ano_base")["inde"].apply(lambda s: int(s.isna().sum())))

print("\nNulos de pedra por ano:")
print(df_long.groupby("ano_base")["pedra"].apply(lambda s: int(s.isna().sum())))


ValueError: Duplicate column names found: ['ra', 'ano_base', 'fase', 'fase_num', 'turma', 'genero', 'instituicao_de_ensino', 'inde', 'pedra', 'iaa', 'ieg', 'ips', 'ida', 'ipv', 'ian', 'ipp', 'nota_mat', 'nota_port', 'nota_ing', 'defasagem', 'fase_ideal', 'fase_ideal', 'ativo_inativo', 'rec_psicologia', 'rec_av1', 'rec_av2', 'rec_av3', 'rec_av4', 'destaque_ieg', 'destaque_ida', 'destaque_ipv', 'sem_inde', 'sem_pedra']

In [1]:
import os
import re
import pandas as pd
import numpy as np
import unicodedata

df = df_fase5.copy()

# 1) Remover coluna 100% vazia (Destaque IPV.1)
if "Destaque IPV.1" in df.columns:
    df = df.drop(columns=["Destaque IPV.1"])

# 2) Corrigir tipos suspeitos
# 2.1 Data de Nasc -> datetime (formatos mistos)
if "Data de Nasc" in df.columns:
    df["Data de Nasc"] = pd.to_datetime(df["Data de Nasc"], errors="coerce")

# 2.2 Idade -> inteiro (corrige casos tipo 1900-01-08)
if "Idade" in df.columns:
    s = df["Idade"]
    idade_num = pd.to_numeric(s, errors="coerce")
    idade_dt = pd.to_datetime(s, errors="coerce")
    idade_from_date = np.where(
        idade_dt.notna() & (idade_dt.dt.year == 1900) & (idade_dt.dt.month == 1),
        idade_dt.dt.day,
        np.nan
    )
    df["Idade"] = pd.Series(idade_num).fillna(pd.Series(idade_from_date)).astype("Int64")

# 2.3 INDE 2024 -> float
if "INDE 2024" in df.columns:
    df["INDE 2024"] = pd.to_numeric(df["INDE 2024"], errors="coerce")

# 3) Harmonizar nomes para snake_case (sem acento/sem espaço)
def norm_col(c):
    c = str(c).strip()
    c = unicodedata.normalize("NFKD", c).encode("ascii", "ignore").decode("ascii")
    c = c.replace("/", "_").replace("-", "_")
    c = "_".join(c.split())
    c = c.replace("__", "_").lower()
    return c

df.columns = [norm_col(c) for c in df.columns]

# 4) Unificar colunas duplicadas do mesmo conceito (coalesce)
def coalesce(df_in, new_col, candidates):
    exist = [c for c in candidates if c in df_in.columns]
    if not exist:
        return df_in
    df_in[new_col] = df_in[exist].bfill(axis=1).iloc[:, 0]
    drop_cols = [c for c in exist if c != new_col]
    if drop_cols:
        df_in = df_in.drop(columns=drop_cols)
    return df_in

df = coalesce(df, "fase_ideal", ["fase_ideal"])          # já deve virar fase_ideal após normalização
df = coalesce(df, "fase_ideal", ["fase_ideal", "fase_ideal_1"])  # segurança, se aparecer variação

df = coalesce(df, "defasagem", ["defas", "defasagem"])

df = coalesce(df, "nota_mat", ["matem", "mat"])
df = coalesce(df, "nota_port", ["portug", "por"])
df = coalesce(df, "nota_ing", ["ingles", "ing"])

# 5) Padronizar medidas por ano
df = coalesce(df, "inde_2022", ["inde_22"])
df = coalesce(df, "inde_2023", ["inde_2023", "inde_23"])
df = coalesce(df, "inde_2024", ["inde_2024"])

df = coalesce(df, "pedra_2022", ["pedra_22"])
df = coalesce(df, "pedra_2023", ["pedra_2023", "pedra_23"])
df = coalesce(df, "pedra_2024", ["pedra_2024"])

# 6) Criar fase e fase_num (padronizado)
if "fase" not in df.columns and "fase" in df.columns:
    pass
if "fase" in df.columns:
    df["fase"] = df["fase"].astype(str).str.strip().str.upper()
    df["fase_num"] = pd.to_numeric(df["fase"].str.extract(r"(\d+)", expand=False), errors="coerce").astype("Int64")

# 7) Garantir tipos numéricos em colunas-chave
for c in ["inde_2022","inde_2023","inde_2024","nota_mat","nota_port","nota_ing","defasagem"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# 8) Blindagem parquet: object -> string
obj_cols = df.select_dtypes(include=["object"]).columns
df[obj_cols] = df[obj_cols].astype("string")

# 9) Salvar processed wide
os.makedirs("data/processed", exist_ok=True)
df.to_parquet("data/processed/pede_2022_2024_processed_wide.parquet", index=False)
df.to_csv("data/processed/pede_2022_2024_processed_wide.csv", index=False, encoding="utf-8")

print("OK. Salvo em data/processed/")
print("Shape:", df.shape)
print("Colunas (amostra):", list(df.columns)[:25])


NameError: name 'df_fase5' is not defined

In [ ]:
import re
import os
import pandas as pd
import numpy as np

# usa df se existir, senão usa df_fase5
try:
    df
except NameError:
    df = df_fase5.copy()

# 1) garantir fase como string padronizada
if "fase" not in df.columns and "Fase" in df.columns:
    df = df.rename(columns={"Fase": "fase"})

df["fase"] = df["fase"].astype(str).str.strip().str.upper()

# 2) criar fase_num (extrai o primeiro número que aparecer)
df["fase_num"] = pd.to_numeric(df["fase"].str.extract(r"(\d+)", expand=False), errors="coerce").astype("Int64")

# 3) auditoria rápida (pra você bater o olho)
print("Top valores de fase:")
print(df["fase"].value_counts(dropna=False).head(20))

print("\nAmostra fase -> fase_num:")
display(df[["fase", "fase_num"]].drop_duplicates().sort_values(["fase_num", "fase"]).head(30))

print("\nQtd fase_num nula:", int(df["fase_num"].isna().sum()))

# 4) blindagem parquet: toda coluna object vira string
obj_cols = df.select_dtypes(include=["object"]).columns
df[obj_cols] = df[obj_cols].astype("string")

# 5) salvar
os.makedirs("data/processed", exist_ok=True)
df.to_parquet("data/processed/pede_2022_2024_processed.parquet", index=False)
df.to_csv("data/processed/pede_2022_2024_processed.csv", index=False, encoding="utf-8")

print("\nOK. Salvo em data/processed/")
print("Shape:", df.shape)


Top valores de fase:
fase
ALFA      427
FASE 2    200
1         192
0         190
FASE 1    173
2         155
3         148
FASE 3    132
FASE 4     94
4          76
FASE 5     65
FASE 8     63
5          60
9          38
FASE 6     33
7E         25
FASE 7     23
8E         23
7          21
6          18
Name: count, dtype: int64

Amostra fase -> fase_num:


,fase,fase_num
670,0,0
478,1,1
2070,1A,1
2084,1B,1
2099,1C,1
2113,1D,1
2123,1E,1
2131,1G,1
2147,1H,1
2159,1J,1



Qtd fase_num nula: 427

OK. Salvo em data/processed/
Shape: (3030, 56)


In [ ]:
# 1) fase_num nula (provável ALFA)
print("fase_num nula:", int(df["fase_num"].isna().sum()))

# 2) sanity check: inde por ano (quando você já tiver inde no LONG, isso vira obrigatório)
print(df_fase5.groupby("ano_base")["RA"].nunique())

# 3) status ativo/inativo (se ainda estiver com os nomes originais)
cols_status = [c for c in df_fase5.columns if "Ativo/ Inativo" in c]
print("Cols status:", cols_status)


fase_num nula: 427
ano_base
2022     860
2023    1014
2024    1156
Name: RA, dtype: int64
Cols status: ['Ativo/ Inativo', 'Ativo/ Inativo.1']


In [ ]:
import pandas as pd

# 1) criar uma coluna única
df_fase5["Ativo_Inativo"] = (
    df_fase5[["Ativo/ Inativo", "Ativo/ Inativo.1"]]
    .bfill(axis=1)
    .iloc[:, 0]
)

# 2) remover as duplicadas
df_fase5 = df_fase5.drop(columns=["Ativo/ Inativo", "Ativo/ Inativo.1"])

# 3) conferência
print("Colunas de status restantes:",
      [c for c in df_fase5.columns if "Ativo" in c or "Inativo" in c])

print("\nTop valores de Ativo_Inativo:")
print(df_fase5["Ativo_Inativo"].astype(str).str.strip().value_counts(dropna=False).head(20))


Colunas de status restantes: ['Ativo_Inativo']

Top valores de Ativo_Inativo:
Ativo_Inativo
nan         1874
Cursando    1156
Name: count, dtype: int64


/tmp/ipython-input-3119124754.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .bfill(axis=1)


**Bloco LONG: vou padronizar a base PEDE (2022–2024) em um único dataset por ano**

Agora vou rodar um bloco, denominado **Bloco LONG**, para transformar a base que hoje está “wide” (com colunas diferentes dependendo do ano) em um formato **LONG**, onde cada linha representa um aluno (**RA**) em um ano específico (**ano_base**) com colunas padronizadas (mesmo nome e mesmo significado para todos os anos).

Meu objetivo aqui é parar de lidar com colunas espalhadas como **INDE 22**, **INDE 23**, **INDE 2023**, **INDE 2024** e passar a ter uma coluna única **inde** (e o mesmo para **pedra**, notas, defasagem etc.). Isso deixa a base pronta para EDA, feature engineering, modelagem e para o Streamlit.

**O que eu vou fazer dentro do Bloco LONG:**

1) **Vou criar e padronizar as colunas fase e fase_num**  
- Vou criar **fase** como texto padronizado (maiúsculo e sem espaços sobrando).  
- Vou criar **fase_num** extraindo o primeiro número encontrado em **fase**.  
  - Exemplos: **FASE 2 → 2**, **1A → 1**, **7E → 7**, **ALFA → vazio (NaN)**.  
- Vou manter **fase** e **fase_num** ao mesmo tempo, porque **fase** pode carregar informação extra (como ALFA e sufixos A/B/E) que eu posso precisar depois.

2) **Vou escolher o INDE correto conforme o ano_base e criar a coluna inde**  
- Para 2022, vou pegar **INDE 22**.  
- Para 2023, vou tentar **INDE 2023** e, se estiver vazio, vou completar com **INDE 23**.  
- Para 2024, vou pegar **INDE 2024**.  
- No fim, vou converter **inde** para numérico.

3) **Vou escolher a Pedra correta conforme o ano_base e criar a coluna pedra**  
- Para 2022, vou pegar **Pedra 22**.  
- Para 2023, vou tentar **Pedra 2023** e completar com **Pedra 23** se precisar.  
- Para 2024, vou pegar **Pedra 2024**.

4) **Vou padronizar as notas escolares**  
Vou criar colunas únicas: **nota_mat**, **nota_port**, **nota_ing**, escolhendo a coluna certa por ano:  
- 2022: **Matem**, **Portug**, **Inglês**  
- 2023/2024: **Mat**, **Por**, **Ing**  
Depois vou converter essas notas para numérico.

5) **Vou padronizar defasagem e fase_ideal**  
- Vou criar **defasagem** usando **Defas** (2022) e **Defasagem** (2023/2024), e vou converter para numérico.  
- Vou criar **fase_ideal** usando **Fase ideal** (2022) e **Fase Ideal** (2023/2024).

6) **Vou converter indicadores para um formato consistente (quando existirem)**  
Quando as colunas existirem na base, vou converter para numérico e salvar com nome em minúsculo:  
**iaa**, **ieg**, **ips**, **ida**, **ipv**, **ian**, **ipp**.

7) **Vou padronizar campos de texto (recomendações e destaques)**  
Quando existirem, vou criar colunas de texto e preencher vazios com string vazia, para ficar pronto para NLP depois:  
**rec_psicologia**, **rec_av1**, **rec_av2**, **rec_av3**, **rec_av4**, **destaque_ieg**, **destaque_ida**, **destaque_ipv**.

8) **Vou incluir o status ativo/inativo (quando existir)**  
Vou levar para o LONG a coluna **ativo_inativo** (no seu caso, aparece principalmente em 2024) e vou normalizar vazios para NaN.

9) **Vou montar o dataset final df_long**  
No final, eu vou selecionar apenas as colunas padronizadas essenciais:  
- Identificação: **RA**, **ano_base**  
- Contexto: **Turma**, **Gênero**, **fase**, **fase_num**  
- Indicadores: **inde**, **pedra**, notas, defasagem, fase_ideal, índices  
- Textos: recomendações e destaques  
- Status: **ativo_inativo** (quando houver)

10) **Vou blindar e salvar em data/processed**  
Antes de salvar, vou converter colunas de texto para **string** (isso evita erro no parquet).  
Depois vou salvar dois arquivos:  
- **data/processed/pede_long_2022_2024.parquet**  
- **data/processed/pede_long_2022_2024.csv**

**Como eu vou validar se deu certo**  
Ao final do bloco, eu vou olhar:  
- o **shape** do df_long  
- quantos nulos ficaram em **inde** por ano_base  
- quantos nulos ficaram em **pedra** por ano_base  

Se 2023 vier com muitos nulos em **inde** ou **pedra**, eu ajusto o mapeamento adicionando as colunas alternativas que ainda não foram consideradas.


In [ ]:
import os
import pandas as pd
import numpy as np

df = df_fase5.copy()
df["ano_base"] = df["ano_base"].astype(int)

# fase e fase_num
df["fase"] = df["Fase"].astype(str).str.strip().str.upper()
df["fase_num"] = pd.to_numeric(df["fase"].str.extract(r"(\d+)", expand=False), errors="coerce").astype("Int64")

def pick_by_year(ano_series, mapping):
    out = pd.Series([pd.NA] * len(ano_series), index=ano_series.index, dtype="object")
    for ano, col in mapping.items():
        if col in df.columns:
            mask = ano_series == ano
            out.loc[mask] = df.loc[mask, col]
    return out

# INDE
df["inde"] = pick_by_year(df["ano_base"], {2022: "INDE 22", 2023: "INDE 2023", 2024: "INDE 2024"})
if "INDE 23" in df.columns:
    m = df["ano_base"] == 2023
    df.loc[m, "inde"] = df.loc[m, "inde"].fillna(df.loc[m, "INDE 23"])
df["inde"] = pd.to_numeric(df["inde"], errors="coerce")

# Pedra
df["pedra"] = pick_by_year(df["ano_base"], {2022: "Pedra 22", 2023: "Pedra 2023", 2024: "Pedra 2024"})
if "Pedra 23" in df.columns:
    m = df["ano_base"] == 2023
    df.loc[m, "pedra"] = df.loc[m, "pedra"].fillna(df.loc[m, "Pedra 23"])

# Notas
df["nota_mat"] = pick_by_year(df["ano_base"], {2022: "Matem", 2023: "Mat", 2024: "Mat"})
df["nota_port"] = pick_by_year(df["ano_base"], {2022: "Portug", 2023: "Por", 2024: "Por"})
df["nota_ing"] = pick_by_year(df["ano_base"], {2022: "Inglês", 2023: "Ing", 2024: "Ing"})

for c in ["nota_mat", "nota_port", "nota_ing"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Defasagem e fase_ideal
df["defasagem"] = pick_by_year(df["ano_base"], {2022: "Defas", 2023: "Defasagem", 2024: "Defasagem"})
df["defasagem"] = pd.to_numeric(df["defasagem"], errors="coerce")

df["fase_ideal"] = pick_by_year(df["ano_base"], {2022: "Fase ideal", 2023: "Fase Ideal", 2024: "Fase Ideal"})

# Indicadores (quando existirem)
for col in ["IAA", "IEG", "IPS", "IDA", "IPV", "IAN", "IPP"]:
    if col in df.columns:
        df[col.lower()] = pd.to_numeric(df[col], errors="coerce")

# Textos (quando existirem)
txt_map = {
    "Rec Psicologia": "rec_psicologia",
    "Rec Av1": "rec_av1",
    "Rec Av2": "rec_av2",
    "Rec Av3": "rec_av3",
    "Rec Av4": "rec_av4",
    "Destaque IEG": "destaque_ieg",
    "Destaque IDA": "destaque_ida",
    "Destaque IPV": "destaque_ipv",
}
for src, dst in txt_map.items():
    if src in df.columns:
        df[dst] = df[src].fillna("").astype(str)

# Status (2024 tem, 22/23 ficam NaN)
if "Ativo_Inativo" in df.columns:
    df["ativo_inativo"] = df["Ativo_Inativo"].astype(str).str.strip()
    df.loc[df["ativo_inativo"].str.lower().isin(["nan", "none", ""]), "ativo_inativo"] = pd.NA

# LONG final
cols_long = [
    "RA", "ano_base", "fase", "fase_num", "Turma", "Gênero",
    "inde", "pedra",
    "iaa", "ieg", "ips", "ida", "ipv", "ian", "ipp",
    "nota_mat", "nota_port", "nota_ing",
    "defasagem", "fase_ideal",
    "ativo_inativo",
    "rec_psicologia", "rec_av1", "rec_av2", "rec_av3", "rec_av4",
    "destaque_ieg", "destaque_ida", "destaque_ipv"
]
cols_long = [c for c in cols_long if c in df.columns]

df_long = df[cols_long].copy()

# blindagem parquet: object -> string
obj_cols = df_long.select_dtypes(include=["object"]).columns
df_long[obj_cols] = df_long[obj_cols].astype("string")

os.makedirs("data/processed", exist_ok=True)
df_long.to_parquet("data/processed/pede_long_2022_2024.parquet", index=False)
df_long.to_csv("data/processed/pede_long_2022_2024.csv", index=False, encoding="utf-8")

print("OK. LONG salvo em data/processed/")
print("Shape:", df_long.shape)

print("\nNulos de inde por ano:")
print(df_long.groupby("ano_base")["inde"].apply(lambda s: int(s.isna().sum())))

print("\nNulos de pedra por ano:")
print(df_long.groupby("ano_base")["pedra"].apply(lambda s: int(s.isna().sum())))


OK. LONG salvo em data/processed/
Shape: (3030, 29)

Nulos de inde por ano:
ano_base
2022      0
2023     83
2024    102
Name: inde, dtype: int64

Nulos de pedra por ano:
ano_base
2022     0
2023    83
2024    64
Name: pedra, dtype: int64


/tmp/ipython-input-1586998363.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[m, "inde"] = df.loc[m, "inde"].fillna(df.loc[m, "INDE 23"])


In [ ]:
import pandas as pd

# irei usar o df original e o df_long que acabei de criar
m2023 = (df_long["ano_base"] == 2023)
m2024 = (df_long["ano_base"] == 2024)

print("2023: inde nulo:", int(df_long.loc[m2023, "inde"].isna().sum()),
      "| pedra nulo:", int(df_long.loc[m2023, "pedra"].isna().sum()))

print("2024: inde nulo:", int(df_long.loc[m2024, "inde"].isna().sum()),
      "| pedra nulo:", int(df_long.loc[m2024, "pedra"].isna().sum()))

# 1) Ver se inde e pedra nulos são as MESMAS linhas (principalmente 2023)
n2023 = df_long.loc[m2023, ["inde","pedra"]].isna()
print("\n2023: nulos em ambos (inde e pedra):", int((n2023["inde"] & n2023["pedra"]).sum()))

# 2) Pegar amostra dos casos nulos em 2023 (pra achar coluna fonte)
idx_2023 = df_long.index[m2023 & (df_long["inde"].isna() | df_long["pedra"].isna())]
amostra_2023 = df_fase5.loc[idx_2023, ["RA","ano_base","INDE 2023","INDE 23","Pedra 2023","Pedra 23"]].head(30)
print("\nAmostra 2023 com nulos:")
display(amostra_2023)

# 3) Amostra dos casos nulos em 2024
idx_2024 = df_long.index[m2024 & (df_long["inde"].isna() | df_long["pedra"].isna())]
amostra_2024 = df_fase5.loc[idx_2024, ["RA","ano_base","INDE 2024","Pedra 2024","IAN"]].head(30)
print("\nAmostra 2024 com nulos:")
display(amostra_2024)


2023: inde nulo: 83 | pedra nulo: 83
2024: inde nulo: 102 | pedra nulo: 64

2023: nulos em ambos (inde e pedra): 83

Amostra 2023 com nulos:


,RA,ano_base,INDE 2023,INDE 23,Pedra 2023,Pedra 23
1414,RA-1126,2023,NaN,NaN,NaN,NaN
1649,RA-145,2023,NaN,NaN,NaN,NaN
1650,RA-1188,2023,NaN,NaN,NaN,NaN
1651,RA-265,2023,NaN,NaN,NaN,NaN
1652,RA-254,2023,NaN,NaN,NaN,NaN
1653,RA-267,2023,NaN,NaN,NaN,NaN
1654,RA-1189,2023,NaN,NaN,NaN,NaN
1798,RA-1225,2023,NaN,NaN,NaN,NaN
1799,RA-76,2023,NaN,NaN,NaN,NaN
1800,RA-26,2023,NaN,NaN,NaN,NaN



Amostra 2024 com nulos:


,RA,ano_base,INDE 2024,Pedra 2024,IAN
2928,RA-1236,2024,NaN,NaN,10.0
2929,RA-1234,2024,NaN,NaN,10.0
2930,RA-1633,2024,NaN,NaN,10.0
2931,RA-1248,2024,NaN,NaN,10.0
2932,RA-1246,2024,NaN,NaN,10.0
2933,RA-1245,2024,NaN,NaN,10.0
2934,RA-1244,2024,NaN,NaN,10.0
2935,RA-1634,2024,NaN,NaN,10.0
2936,RA-1242,2024,NaN,NaN,10.0
2937,RA-1250,2024,NaN,NaN,10.0


In [ ]:
import os
import pandas as pd
import numpy as np

# carrega o LONG que você acabou de salvar (ou use df_long se estiver em memória)
df_long = df_long.copy()

df_long["sem_inde"] = df_long["inde"].isna()
df_long["sem_pedra"] = df_long["pedra"].isna()

print("Sem INDE por ano:")
print(df_long.groupby("ano_base")["sem_inde"].sum())

print("\nSem Pedra por ano:")
print(df_long.groupby("ano_base")["sem_pedra"].sum())

# versão para treino/EDA que exige inde/pedra
df_model = df_long.loc[~df_long["sem_inde"]].copy()

print("\nShape df_long:", df_long.shape)
print("Shape df_model (sem inde nulo):", df_model.shape)

os.makedirs("data/processed", exist_ok=True)
df_model.to_parquet("data/processed/pede_long_2022_2024_model.parquet", index=False)
df_model.to_csv("data/processed/pede_long_2022_2024_model.csv", index=False, encoding="utf-8")

print("\nOK. Arquivo para modelagem salvo em data/processed/")


Sem INDE por ano:
ano_base
2022      0
2023     83
2024    102
Name: sem_inde, dtype: int64

Sem Pedra por ano:
ano_base
2022     0
2023    83
2024    64
Name: sem_pedra, dtype: int64

Shape df_long: (3030, 31)
Shape df_model (sem inde nulo): (2845, 31)

OK. Arquivo para modelagem salvo em data/processed/


In [ ]:
import pandas as pd
import numpy as np

# carrega o model se precisar
# df_model = pd.read_parquet("data/processed/pede_long_2022_2024_model.parquet")

df_model = df_model.copy()

# 1) flags para fase
df_model["fase_alfa"] = df_model["fase"].astype(str).str.upper().eq("ALFA")
df_model["fase_tipo"] = np.where(df_model["fase_alfa"], "ALFA", "NUMERICA")

print("Qtd ALFA no df_model:", int(df_model["fase_alfa"].sum()))

# 2) faltantes por colunas-chave
cols_check = ["pedra","nota_mat","nota_port","nota_ing","iaa","ieg","ips","ida","ipv","ian","ipp","defasagem","fase_ideal"]
cols_check = [c for c in cols_check if c in df_model.columns]

falt = df_model[cols_check].isna().sum().sort_values(ascending=False)
print("\nFaltantes (top):")
print(falt.head(15))

# 3) salvar versão final saneada para modelagem
df_model.to_parquet("data/processed/pede_long_2022_2024_model_v2.parquet", index=False)
df_model.to_csv("data/processed/pede_long_2022_2024_model_v2.csv", index=False, encoding="utf-8")

print("\nOK. Salvo model_v2 em data/processed/")
print("Shape:", df_model.shape)



Qtd ALFA no df_model: 427

Faltantes (top):
nota_ing      1761
ipp            860
nota_port        7
nota_mat         6
pedra            0
iaa              0
ieg              0
ida              0
ips              0
ipv              0
ian              0
defasagem        0
fase_ideal       0
dtype: int64

OK. Salvo model_v2 em data/processed/
Shape: (2845, 33)


In [ ]:
print("Nulos por ano (nota_ing):")
print(df_model.groupby("ano_base")["nota_ing"].apply(lambda s: int(s.isna().sum())))

print("\nNulos por ano (ipp):")
print(df_model.groupby("ano_base")["ipp"].apply(lambda s: int(s.isna().sum())) if "ipp" in df_model.columns else "ipp não existe")


Nulos por ano (nota_ing):
ano_base
2022    577
2023    603
2024    581
Name: nota_ing, dtype: int64

Nulos por ano (ipp):
ano_base
2022    860
2023      0
2024      0
Name: ipp, dtype: int64


In [ ]:
import os
import pandas as pd
import numpy as np

df_model_v2 = df_model.copy()

# flags
df_model_v2["tem_nota_ing"] = df_model_v2["nota_ing"].notna()
df_model_v2["tem_ipp"] = df_model_v2["ipp"].notna() if "ipp" in df_model_v2.columns else False

# baseline features: remove colunas problemáticas por missing
drop_cols = []
if "nota_ing" in df_model_v2.columns: drop_cols.append("nota_ing")
if "ipp" in df_model_v2.columns: drop_cols.append("ipp")

df_model_v2 = df_model_v2.drop(columns=drop_cols)

# (opcional) eliminar as poucas linhas com nota_mat/nota_port nulas
for c in ["nota_mat", "nota_port"]:
    if c in df_model_v2.columns:
        df_model_v2 = df_model_v2[df_model_v2[c].notna()]

print("Shape final (v1):", df_model_v2.shape)

os.makedirs("data/processed", exist_ok=True)
df_model_v2.to_parquet("data/processed/pede_long_2022_2024_model_v3.parquet", index=False)
df_model_v2.to_csv("data/processed/pede_long_2022_2024_model_v3.csv", index=False, encoding="utf-8")

print("OK. Salvo model_v3 em data/processed/")


Shape final (v1): (2834, 33)
OK. Salvo model_v3 em data/processed/
